# Grid Search CV

In [ ]:
import numpy as np
import pandas as pd

# Модель для прогнозирования, например ARIMA
from sktime.forecasting.arima import ARIMA
# Визуализация временных рядов
from sktime.utils.plotting import plot_series
# Модули для кросс-валидации
from sktime.split import temporal_train_test_split, ExpandingWindowSplitter, SlidingWindowSplitter, SingleWindowSplitter
from sktime.forecasting.model_evaluation import evaluate
from sktime.performance_metrics.forecasting import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError # Метрики MSE, MAE, MAPE
# Поиск оптимальных гиперпараметров по сетке
from sktime.forecasting.model_selection import ForecastingGridSearchCV

import pandas_datareader.data as web

# настройки визуализации
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) месячные данные по Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity (Symbol [`GS10`](https://fred.stlouisfed.org/series/GS10)) с 2000-01-01 по н.в. и создадим датафрейм `y`

In [ ]:
y = web.DataReader(name='GS10', data_source='fred', start='2000-01')
y.index = y.index.to_period(freq='M')

Поиск оптимальных (гипер)парамеров модели по стеке

In [ ]:
# Зададим метод прогнозирования
forecaster = ARIMA()

# разбиваем параметры кросс-валидации
cv_strategy = ExpandingWindowSplitter(fh=np.arange(1, 6), initial_window=100, step_length=5)

# Задаём сетку для значений параметров модели в виде словаря
# будем менять параметры order и trend у ARIMA
# Важно: будут перебираться всевозможные комбинации!
param_grid = {'order':[(0,1,0), (1,1,0), (0,1,1)], 'trend': ['c', 'n']}

# инициализируем метрики
metric = MeanSquaredError(square_root=False)

# Grid search
gscv = ForecastingGridSearchCV(forecaster=forecaster, param_grid=param_grid, cv=cv_strategy, scoring=metric)

gscv.fit(y)

In [ ]:
# Параметры оптимальной модели
gscv.get_fitted_params()['best_forecaster']